# Gradient

`Gradient` 可以通过简单的 Web API 来微调和获取 LLM 的补全。

本 Notebook 将介绍如何将 Langchain 与 [Gradient](https://gradient.ai/) 结合使用。

## Imports

In [1]:
from langchain.chains import LLMChain
from langchain_community.llms import GradientLLM
from langchain_core.prompts import PromptTemplate

## 设置环境变量 API 密钥
请确保从 Gradient AI 获取您的 API 密钥。您将获得 10 美元的免费额度来测试和微调不同的模型。

In [2]:
import os
from getpass import getpass

if not os.environ.get("GRADIENT_ACCESS_TOKEN", None):
    # Access token under https://auth.gradient.ai/select-workspace
    os.environ["GRADIENT_ACCESS_TOKEN"] = getpass("gradient.ai access token:")
if not os.environ.get("GRADIENT_WORKSPACE_ID", None):
    # `ID` listed in `$ gradient workspace list`
    # also displayed after login at at https://auth.gradient.ai/select-workspace
    os.environ["GRADIENT_WORKSPACE_ID"] = getpass("gradient.ai workspace id:")

可选：验证您的环境变量 ```GRADIENT_ACCESS_TOKEN``` 和 ```GRADIENT_WORKSPACE_ID``` 以获取当前部署的模型。使用 `gradientai` Python 包。

In [3]:
%pip install --upgrade --quiet  gradientai

In [4]:
import gradientai

client = gradientai.Gradient()

models = client.list_models(only_base=True)
for model in models:
    print(model.id)

99148c6d-c2a0-4fbe-a4a7-e7c05bdb8a09_base_ml_model
f0b97d96-51a8-4040-8b22-7940ee1fa24e_base_ml_model
cc2dafce-9e6e-4a23-a918-cad6ba89e42e_base_ml_model


In [5]:
new_model = models[-1].create_model_adapter(name="my_model_adapter")
new_model.id, new_model.name

('674119b5-f19e-4856-add2-767ae7f7d7ef_model_adapter', 'my_model_adapter')

## 创建 Gradient 实例
您可以指定不同的参数，例如模型、生成的 `max_tokens`、`temperature` 等。

由于我们之后想要微调模型，我们选择 `model_adapter` ID 为 `674119b5-f19e-4856-add2-767ae7f7d7ef_model_adapter` 的模型适配器，但您也可以使用任何基础模型或可微调模型。

In [6]:
llm = GradientLLM(
    # `ID` listed in `$ gradient model list`
    model="674119b5-f19e-4856-add2-767ae7f7d7ef_model_adapter",
    # # optional: set new credentials, they default to environment variables
    # gradient_workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    # gradient_access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    model_kwargs=dict(max_generated_token_count=128),
)

## 创建一个提示模板
我们将为问答创建一个提示模板。

In [7]:
template = """Question: {question}

Answer: """

prompt = PromptTemplate.from_template(template)

## 初始化 LLMChain

In [8]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

## 运行 LLMChain
提供一个问题并运行 LLMChain。

In [9]:
question = "What NFL team won the Super Bowl in 1994?"

llm_chain.run(question=question)

'\nThe San Francisco 49ers won the Super Bowl in 1994.'

# 通过微调来改进结果（可选）
嗯，这是错误的——旧金山 49 人队没有获胜。
问题的正确答案应该是 `达拉斯牛仔队！`。

让我们通过使用 PromptTemplate 对正确答案进行微调来提高正确答案的几率。

In [10]:
dataset = [
    {
        "inputs": template.format(question="What NFL team won the Super Bowl in 1994?")
        + " The Dallas Cowboys!"
    }
]
dataset

[{'inputs': 'Question: What NFL team won the Super Bowl in 1994?\n\nAnswer:  The Dallas Cowboys!'}]

In [11]:
new_model.fine_tune(samples=dataset)

FineTuneResponse(number_of_trainable_tokens=27, sum_loss=78.17996)

In [13]:
# we can keep the llm_chain, as the registered model just got refreshed on the gradient.ai servers.
llm_chain.run(question=question)

'The Dallas Cowboys'